# Song Emotion Classification, Using Multinomial Naive Bayes, and TF feature extraction

By:

Anak Agung Gede Ngurah Ananda Wirasena (2308561079)

Akira Rian Satya Dhamma (2308561079)

I Gede Abhijana Prayata Wistara (2308561142)

Benedictus Albert Effendi (2308561055)

## 1. Import Libraries yang Digunakan 

In [27]:
from collections import defaultdict
import nltk
import re
import pandas as pd
import numpy as np
# from deep_translator import GoogleTranslator


## 2. Definisikan Parameter Global yang Akan Digunakan, serta Loading Dataset Utama

In [28]:
tempTargetFilename =  ['./data/indo-song-emot.csv', './data/augmented.csv']
targetFilename = tempTargetFilename[0]

resultFilename = ['./data/train-data.csv', './data/test-data.csv'] # Index 0 untuk train, 1 untuk test

# dataSplit = [0.7, 0.3] # Bisa memilih pembagian data 80%-20% atau 70%-30%
dataSplit = [0.8, 0.2] 
fold = 10 # Banyak fold untuk k-fold cross validation
emotion_label = ['bahagia', 'sedih', 'marah', 'takut']

# translator = GoogleTranslator(source='id', target='en')

base_dataset = pd.read_csv('./data/augmented.csv', index_col=0) #mengambil csv menjadi pandas, dan mendefinisikan kolom pertama sebagai index
column_csvs = base_dataset.columns.tolist() # simpen nama kolom untuk training dan testing nanti
base_dataset.head()


,title,artist,full,reff,emotion
0,Augmented Title - bahagia 1,Augmented Artist,lihat getirnya melewatkanmu na statusnya peluk...,menggapai lamar manja putih sekuntum maksud se...,bahagia
1,Augmented Title - marah 1,Augmented Artist,realita berpisah sedihmu pergilah lukamu time ...,kusesali puaskah takkan gombalan ukir famili y...,marah
2,Augmented Title - marah 2,Augmented Artist,kulawan gigi serigala asap bermain silau harap...,palsu januari menunggu you kemari bantu he tol...,marah
3,Augmented Title - marah 3,Augmented Artist,wakil love terbangkan kebobolan kuhapus kuingi...,beralaskan segenggam menyingkirkan hantamlah m...,marah
4,Augmented Title - marah 4,Augmented Artist,percintaan maksudmu sombongnya hempas runtuh k...,obi dipundak come keinginanmu kumpulan every p...,marah


## 3. Persiapan k-Fold Cross Validation, dan Fungsi Pembagian Data 

### 3.1 Definisi Fungsi Pembagian Data, yakni calcFold()
Fungsi ini memastikan pembagian dari data terdistribusi secara seimbang (as possible), untuk tiap fold nantinya  

In [29]:
def calcFold(pdSeries):
    global fold
    newSeries = pd.Series(dtype=object)

    for idx, value in pdSeries.items():
        tempList = [int(value / fold)] * fold
        tempRemainder = value % fold

        for i in range(tempRemainder):
            tempList[-(i + 1)] += 1

        newSeries.loc[idx] = tempList

    return newSeries

### 3.2 Menghitung Distribusi dari Tiap Label Emosi

In [30]:
labels_count = base_dataset['emotion'].value_counts()
print(labels_count)

print("\n<========After Folding==========>")
foldedLabels = calcFold(labels_count)
print(foldedLabels)

emotion
sedih      151
bahagia    131
marah       52
takut       38
Name: count, dtype: int64

<========After Folding==========>
sedih      [15, 15, 15, 15, 15, 15, 15, 15, 15, 16]
bahagia    [13, 13, 13, 13, 13, 13, 13, 13, 13, 14]
marah                [5, 5, 5, 5, 5, 5, 5, 5, 6, 6]
takut                [3, 3, 4, 4, 4, 4, 4, 4, 4, 4]
dtype: object


### 3.3 Membagi Data Sesuai Kalkulasi Distribusinya, untuk Tiap Fold

In [31]:
foldedList = defaultdict(list) # Dictionary dengan struktur {fold: [list for tiap row data]}, untuk menyimpan masing-masing fold
foldPointer = {emotion: 0 for emotion in emotion_label} # Pointer untuk membantu assign row data tertentu pada fold yang belum "penuh"

for idx, data in base_dataset.iterrows(): # data is a pd.Series
    emot = data['emotion']
    fPE = foldPointer[emot] #variabel untuk menyimpan nilai yang relevan agar tidak dipanggil setiap kali dibutuhan
    
    # append row data ke dictionary penyimpan fold
    foldedList[fPE].append(list(data.values))
    
    # decrement untuk pencatatan
    foldedLabels[emot][fPE] -= 1
    
    # ketika list pada indeks tertentu sudah mencapai 0, artinya telah memenuhi pembagian pada fold tersebut, jadi akan increment pointernya pada emosi tersebut
    if foldedLabels[emot][fPE] == 0:
        if foldPointer[emot] < fold -1: # mencegah index out of bound
            foldPointer[emot] += 1

# keperluan display
foldedList_display_dict = {k: v[:5] for k, v in foldedList.items()} # mengambil 5 indeks pertama saja dari masing-masing fold
foldedListDF_display = pd.DataFrame(dict(foldedList_display_dict))
foldedListDF_display.head()

,0,1,2,3,4,5,6,7,8,9
0,"[Augmented Title - bahagia 1, Augmented Artist...","[Augmented Title - takut 4, Augmented Artist, ...","[galang rambu anarki, iwan fals, galang rambu ...","[bunga citra lestari, kecewa, sedikit waktu ya...","[langit abu-abu, tulus, tak mungkin secepat it...","[lelaki pencemburu, dewa19, detak di setiap de...","[tetap dalam jiwa, isyana sarasvati, tak perna...","[sandaran hati, letto, yakinkah kuberdiri. dia...","[selow, cover by via vallen, sudah biasa. diri...","[kukira kau rumah, amigdala, kau datang tidak ..."
1,"[Augmented Title - marah 1, Augmented Artist, ...","[Augmented Title - takut 5, Augmented Artist, ...","[lantas, juicy luicy, lima hari sudah kurindu....","[hivi!, orang ke 3, saat berjumpa dan kau meny...","[cinta, naura ayu, bergetar hatiku saat kuberk...","[365, tiara andini, tiga ratus enam puluh lima...","[ kulakukan semua untukmu, ran, hanya denganmu...","[bukan superstar, project pop, andai aku pasha...","[sang dewi, lyodra, walaupun jiwaku pernah ter...","[maafkan aku, tiara andini, aku telah tahu kit..."
2,"[Augmented Title - marah 2, Augmented Artist, ...","[pergilah kasih, chrisye, tak pernah kusangka ...","[jatuh suka, tulus, sungguh aku tidak memiliki...","[afgan, jodoh pasti bertemu, andai engkau tahu...","[harta berharga, bunga citra lestari, harta ya...","[menjadi dia, tiara andini, aku ingat waktu ka...","[heavy rotation, jkt, satu.dua.tiga.empat . ak...","[sahabat selamanya, padi, dua tiga kapal belay...","[pikiran dan perjalanan, barasuara, di pikiran...","[dengan caraku, arsy widianto, brisia jodie, t..."
3,"[Augmented Title - marah 3, Augmented Artist, ...","[dia milikku, yovie & nuno, semula aku tidak t...","[bicara, monita tahalea, sudah berapa lama aku...","[gac, bahagia, hai hai apa kabar kawan. siapka...","[pasti bisa, citra scholastika, mentari terben...","[aku yang salah, nuca x mahalini, aku tidak me...","[kamu, coboy junior, kamu buat aku tersipu. bu...","[terlatih patah hati, the rain ft. endank soek...","[konon katanya, kunto aji, kau tidak pernah be...","[nyaman, andmesh, lama sudah aku menanti. bany..."
4,"[Augmented Title - marah 4, Augmented Artist, ...","[bertaut, nadin amizah, bun hidup berjalan sep...","[remaja, hivi, kita remaja yang sedang dimabuk...","[duka, last child, kau membunuhku dengan keped...","[teman bahagia, jazz, takkan pernah terlintas....","[pesan terakhir, lyodra, telah aku coba terus ...","[status palsu, vidi aldiano, separuh hati deng...","[buka semangat baru, ello, barry, ipank, lala ...","[selamat (selamat tinggal), virgoun, audy, di ...","[laskar pelangi, nidji, mimpi adalah kunci. un..."


## 4. Text Preprocessing

### 4.1 Inisialisasi Stopword
Menggunakan beberapa kata tambahan berdasarkan eksplorasi data, serta beberapa kata dari berbagai sumber (githup repo, artikel, dan libray nltk)

In [32]:
from nltk.corpus import stopwords

predef = ["kau","ra","ran","ta","wo","uh","ah","oh","ho","woho","hm","nya","tis","ada","adalah","adanya","adapun","agak","agaknya","agar","akan","akankah","akhir","akhiri","akhirnya","aku","akulah","amat","amatlah","anda","andalah","antar","antara","antaranya","apa","apaan","apabila","apakah","apalagi","apatah","artinya","asal","asalkan","atas","atau","ataukah","ataupun","awal","awalnya","bagai","bagaikan","bagaimana","bagaimanakah","bagaimanapun","bagi","bagian","bahkan","bahwa","bahwasanya","baik","bakal","bakalan","balik","banyak","bapak","baru","bawah","beberapa","begini","beginian","beginikah","beginilah","begitu","begitukah","begitulah","begitupun","bekerja","belakang","belakangan","belum","belumlah","benar","benarkah","benarlah","berada","berakhir","berakhirlah","berakhirnya","berapa","berapakah","berapalah","berapapun","berarti","berawal","berbagai","berdatangan","beri","berikan","berikut","berikutnya","berjumlah","berkali-kali","berkata","berkehendak","berkeinginan","berkenaan","berlainan","berlalu","berlangsung","berlebihan","bermacam","bermacam-macam","bermaksud","bermula","bersama","bersama-sama","bersiap","bersiap-siap","bertanya","bertanya-tanya","berturut","berturut-turut","bertutur","berujar","berupa","besar","betul","betulkah","biasa","biasanya","bila","bilakah","bisa","bisakah","boleh","bolehkah","bolehlah","buat","bukan","bukankah","bukanlah","bukannya","bulan","bung","cara","caranya","cukup","cukupkah","cukuplah","cuma","dahulu","dalam","dan","dapat","dari","daripada","datang","dekat","demi","demikian","demikianlah","dengan","depan","di","dia","diakhiri","diakhirinya","dialah","diantara","diantaranya","diberi","diberikan","diberikannya","dibuat","dibuatnya","didapat","didatangkan","digunakan","diibaratkan","diibaratkannya","diingat","diingatkan","diinginkan","dijawab","dijelaskan","dijelaskannya","dikarenakan","dikatakan","dikatakannya","dikerjakan","diketahui","diketahuinya","dikira","dilakukan","dilalui","dilihat","dimaksud","dimaksudkan","dimaksudkannya","dimaksudnya","diminta","dimintai","dimisalkan","dimulai","dimulailah","dimulainya","dimungkinkan","dini","dipastikan","diperbuat","diperbuatnya","dipergunakan","diperkirakan","diperlihatkan","diperlukan","diperlukannya","dipersoalkan","dipertanyakan","dipunyai","diri","dirinya","disampaikan","disebut","disebutkan","disebutkannya","disini","disinilah","ditambahkan","ditandaskan","ditanya","ditanyai","ditanyakan","ditegaskan","ditujukan","ditunjuk","ditunjuki","ditunjukkan","ditunjukkannya","ditunjuknya","dituturkan","dituturkannya","diucapkan","diucapkannya","diungkapkan","dong","dua","dulu","empat","enggak","enggaknya","entah","entahlah","guna","gunakan","hal","hampir","hanya","hanyalah","hari","harus","haruslah","harusnya","hendak","hendaklah","hendaknya","hingga","ia","ialah","ibarat","ibaratkan","ibaratnya","ibu","ikut","ingat","ingat-ingat","ingin","inginkah","inginkan","ini","inikah","inilah","itu","itukah","itulah","jadi","jadilah","jadinya","jangan","jangankan","janganlah","jauh","jawab","jawaban","jawabnya","jelas","jelaskan","jelaslah","jelasnya","jika","jikalau","juga","jumlah","jumlahnya","justru","kala","kalau","kalaulah","kalaupun","kalian","kami","kamilah","kamu","kamulah","kan","kapan","kapankah","kapanpun","karena","karenanya","kasus","kata","katakan","katakanlah","katanya","ke","keadaan","kebetulan","kecil","kedua","keduanya","keinginan","kelamaan","kelihatan","kelihatannya","kelima","keluar","kembali","kemudian","kemungkinan","kemungkinannya","kenapa","kepada","kepadanya","kesampaian","keseluruhan","keseluruhannya","keterlaluan","ketika","khususnya","kini","kinilah","kira","kira-kira","kiranya","kita","kitalah","kok","kurang","lagi","lagian","lah","lain","lainnya","lalu","lama","lamanya","lanjut","lanjutnya","lebih","lewat","lima","luar","macam","maka","makanya","makin","malah","malahan","mampu","mampukah","mana","manakala","manalagi","masa","masalah","masalahnya","masih","masihkah","masing","masing-masing","mau","maupun","melainkan","melakukan","melalui","melihat","melihatnya","memang","memastikan","memberi","memberikan","membuat","memerlukan","memihak","meminta","memintakan","memisalkan","memperbuat","mempergunakan","memperkirakan","memperlihatkan","mempersiapkan","mempersoalkan","mempertanyakan","mempunyai","memulai","memungkinkan","menaiki","menambahkan","menandaskan","menanti","menanti-nanti","menantikan","menanya","menanyai","menanyakan","mendapat","mendapatkan","mendatang","mendatangi","mendatangkan","menegaskan","mengakhiri","mengapa","mengatakan","mengatakannya","mengenai","mengerjakan","mengetahui","menggunakan","menghendaki","mengibaratkan","mengibaratkannya","mengingat","mengingatkan","menginginkan","mengira","mengucapkan","mengucapkannya","mengungkapkan","menjadi","menjawab","menjelaskan","menuju","menunjuk","menunjuki","menunjukkan","menunjuknya","menurut","menuturkan","menyampaikan","menyangkut","menyatakan","menyebutkan","menyeluruh","menyiapkan","merasa","mereka","merekalah","merupakan","meski","meskipun","meyakini","meyakinkan","minta","mirip","misal","misalkan","misalnya","mula","mulai","mulailah","mulanya","mungkin","mungkinkah","nah","naik","namun","nanti","nantinya","nyaris","nyatanya","oleh","olehnya","pada","padahal","padanya","pak","paling","panjang","pantas","para","pasti","pastilah","penting","pentingnya","per","percuma","perlu","perlukah","perlunya","pernah","persoalan","pertama","pertama-tama","pertanyaan","pertanyakan","pihak","pihaknya","pukul","pula","pun","punya","rasa","rasanya","rata","rupanya","saat","saatnya","saja","sajalah","saling","sama","sama-sama","sambil","sampai","sampai-sampai","sampaikan","sana","sangat","sangatlah","satu","saya","sayalah","se","sebab","sebabnya","sebagai","sebagaimana","sebagainya","sebagian","sebaik","sebaik-baiknya","sebaiknya","sebaliknya","sebanyak","sebegini","sebegitu","sebelum","sebelumnya","sebenarnya","seberapa","sebesar","sebetulnya","sebisanya","sebuah","sebut","sebutlah","sebutnya","secara","secukupnya","sedang","sedangkan","sedemikian","sedikit","sedikitnya","seenaknya","segala","segalanya","segera","seharusnya","sehingga","seingat","sejak","sejauh","sejenak","sejumlah","sekadar","sekadarnya","sekali","sekali-kali","sekalian","sekaligus","sekalipun","sekarang","sekecil","seketika","sekiranya","sekitar","sekitarnya","sekurang-kurangnya","sekurangnya","sela","selagi","selain","selaku","selalu","selama","selama-lamanya","selamanya","selanjutnya","seluruh","seluruhnya","semacam","semakin","semampu","semampunya","semasa","semasih","semata","semata-mata","semaunya","sementara","semisal","semisalnya","sempat","semua","semuanya","semula","sendiri","sendirian","sendirinya","seolah","seolah-olah","seorang","sepanjang","sepantasnya","sepantasnyalah","seperlunya","seperti","sepertinya","sepihak","sering","seringnya","serta","serupa","sesaat","sesama","sesampai","sesegera","sesekali","seseorang","sesuatu","sesuatunya","sesudah","sesudahnya","setelah","setempat","setengah","seterusnya","setiap","setiba","setibanya","setidak-tidaknya","setidaknya","setinggi","seusai","sewaktu","siap","siapa","siapakah","siapapun","sini","sinilah","soal","soalnya","suatu","sudah","sudahkah","sudahlah","supaya","tadi","tadinya","tahu","tahun","tak","tambah","tambahnya","tampak","tampaknya","tandas","tandasnya","tanpa","tanya","tanyakan","tanyanya","tapi","tegas","tegasnya","telah","tempat","tengah","tentang","tentu","tentulah","tentunya","tepat","terakhir","terasa","terbanyak","terdahulu","terdapat","terdiri","terhadap","terhadapnya","teringat","teringat-ingat","terjadi","terjadilah","terjadinya","terkira","terlalu","terlebih","terlihat","termasuk","ternyata","tersampaikan","tersebut","tersebutlah","tertentu","tertuju","terus","terutama","tetap","tetapi","tiap","tiba","tiba-tiba","tidak","tidakkah","tidaklah","tiga","tinggi","toh","tunjuk","turut","tutur","tuturnya","ucap","ucapnya","ujar","ujarnya","umum","umumnya","ungkap","ungkapnya","untuk","usah","usai","waduh","wah","wahai","waktu","waktunya","walau","walaupun","wong","yaitu","yakin","yakni","yang","di","ke","dari","dan","yang"]

stopWords = set(stopwords.words('indonesian'))
stopWords.update(set(predef))
# stopWords.update(set(nuStopWordFromAnalysis))

print(stopWords)

det_word = {'ku', 'kau', 'mu', '',}



# whole_corpus = sorted(whole_corpus)

# print(whole_corpus)

{'meyakinkan', 'seolah-olah', 'memperkirakan', 'apakah', 'mampu', 'masa', 'meskipun', 'berkata', 'dahulu', 'apatah', 'lanjutnya', 'juga', 'pasti', 'bung', 'disebutkan', 'mengucapkan', 'sepertinya', 'berbagai', 'inikah', 'sendirinya', 'mendatangi', 'tandasnya', 'sinilah', 'diketahuinya', 'pun', 'semata-mata', 'terjadilah', 'hm', 'semasa', 'tetapi', 'diibaratkan', 'ditunjukkannya', 'berkehendak', 'serta', 'ditunjuknya', 'minta', 'lagi', 'dimintai', 'lebih', 'tegasnya', 'tadi', 'mengingat', 'akankah', 'dimulailah', 'kedua', 'maupun', 'tandas', 'tutur', 'ditambahkan', 'demikian', 'tiba-tiba', 'lamanya', 'segala', 'apabila', 'jelasnya', 'berikutnya', 'menyiapkan', 'sering', 'meyakini', 'ujarnya', 'ibu', 'beginikah', 'jelaslah', 'diketahui', 'sekiranya', 'keadaan', 'sebagian', 'sedikitnya', 'siapakah', 'seenaknya', 'mengucapkannya', 'segera', 'maka', 'mengerjakan', 'lima', 'mampukah', 'dua', 'merupakan', 'hendaklah', 'sedang', 'oleh', 'semula', 'memulai', 'sekali', 'berada', 'dikatakannya', 

### 4.2. Definisikan Fungsi Helper Preprocessing 
Yakni sebagai berikut:
- `reduceWords`: Menyderhanakan kata-kata ekspresif yang sering muncul pada lirik lagu (e.g., "asmaraaaa" >> "asmara").
- `removeStopWords`: Menghilangkan stopword yang telah diinisialisasi pada bagian sebelumnya.
- `preProcess`: Pipeline utama dalam preprocessing, mulai dari cleaning(menghilangkan karakter non-alfanumerikal), tokenisasi, reduceWords(), dan removeStopWords(). (tambahan nanti untuk sinonim dengan terjemahan)

In [33]:
def reduceWords(words):
    def reduce(word):
        while (True):
            if(len(word) > 1):
                if word[-1] == word[-2]: #ketika huruf di kedua indeks terakhir sama (e.g. rumaHH)
                    word = word[:-1] #maka indeks terakhir akan dihapus
                else:
                    return word #jika sudah berbeda langsung direturn
            else:
                return word 

    newToken = []
    for word in words:
        reduced_word = reduce(str(word)) 
        if reduced_word: 
            newToken.append(reduced_word)

    return newToken

def removeStopWords(token):
    global stopWords
    return [tkn for tkn in token if tkn not in stopWords and tkn] # mengembalikan token yang tidak ada di set stopWords

def preProcess(text):
    pat = r'[^a-zA-Z0-9\s]' # pattern untuk library re untuk menyisakan karakter alfanumerik saja (dengan spasi)

    tokenz = str(text).lower() # memastikan setiap kata bukan huruf kapital, walaupun pada dataset asli sudah bersih, tetapi untuk implementasi ketika meminta input nantinya
    tokenz = re.sub(pat, '', tokenz)  # apply aturan yang sudah diinisialisasi

    # tokenz = stemmer.stem(tokenz)  # stemming saat ini hanya memperlambat dan merusak akurasi

    tokenz = nltk.word_tokenize(tokenz)  # tokenisasi menggunakan library (lebih cepat)

    tokenz = reduceWords(tokenz) 

    tokenz = removeStopWords(tokenz)

    # print("Currently translating " + str(len(tokenz)) + " token")
    # tokenz = translator.translate_batch(tokenz)

    return tokenz

print(preProcess("AkU,, adalAHHH anak GeMBaLaAAaa~")) #contoh implementasi

['anak', 'gembala']


## 5. Multinomial Naive Bayes (MNB) Classifier

### 5.1 Inisialisasi Variabel yang Akan Digunakan, serta Fungsi Helper
Fungsi Helper dengan penjelasan sebagai berikut:
- `countFreq`: Menghitung banyaknya suatu term muncul per label, serta total term pada tiap label.
- `calcCP`: Menghitung conditional probability, dari term terkait emosi tertentu, dengan nilai smoothing terbaik (hypertuning).
- `resetGlobals`: Mereset seluruh variabel global, agar bisa digunakan kembali untuk tiap fold.

In [34]:
tf_given_label_full = defaultdict(lambda: defaultdict(int))  # term frequency untuk 'full' dan 'reff'(dibawah) lirik berdasarkan label tertentu
tf_given_label_reff = defaultdict(lambda: defaultdict(int))  # dict = {'term':{'label':tf}}

number_of_term_at_a_label_full = defaultdict(int)  # total banyak term dari lirik pada label tertentu
number_of_term_at_a_label_reff = defaultdict(int)  # dict = {'label':number of term}

number_of_unique_term_across_label_full = 0  # banyaknya term unik dari seluruh label (seluruh dataset)
number_of_unique_term_across_label_reff = 0

label_frequency = defaultdict(int)  # frekuensi tiap label: dict = {'label':frequency}
total_label_frequency = 0  # Total of frequency from label_frequency

song_unique_term_full = set() # set dari seluruh term unik yang ada pada 'full' dan 'reff' lyric
song_unique_term_reff = set() 

term_conditional_probability_at_label_full = defaultdict(lambda: defaultdict(float))
term_conditional_probability_at_label_reff = defaultdict(lambda: defaultdict(float))

def resetGlobals():
    global tf_given_label_full, tf_given_label_reff
    global number_of_term_at_a_label_full, number_of_term_at_a_label_reff
    global number_of_unique_term_across_label_full, number_of_unique_term_across_label_reff
    global label_frequency, total_label_frequency
    global song_unique_term_full, song_unique_term_reff
    global term_conditional_probability_at_label_full, term_conditional_probability_at_label_reff

    tf_given_label_full = defaultdict(lambda: defaultdict(int))
    tf_given_label_reff = defaultdict(lambda: defaultdict(int))

    number_of_term_at_a_label_full = defaultdict(int)
    number_of_term_at_a_label_reff = defaultdict(int)

    number_of_unique_term_across_label_full = 0
    number_of_unique_term_across_label_reff = 0

    label_frequency = defaultdict(int)
    total_label_frequency = 0

    song_unique_term_full = set()
    song_unique_term_reff = set()

    term_conditional_probability_at_label_full = defaultdict(lambda: defaultdict(float))
    term_conditional_probability_at_label_reff = defaultdict(lambda: defaultdict(float))
    
smoothing = 0.0096 # laplace smoothing alpha value (telah melalui hypertuning)


def countFreq(emo, text, termFreq, termCount):
    for word in text:
        termFreq[emo][word] += 1 # mencari frekuensi kemunculan suatu term(word) pada tiap label(emo)
        termCount[emo] += 1 # menyimpan banyaknya term pada label tertentu

def calcCP(terms, local_emotion_labels, cpDict, tfOLabel, totalTerm, uniqTerm):
    global smoothing
    for term in terms:
        for emo in local_emotion_labels:
            # P(term|class) = (count(term,class) + alpha) / (count(all_terms_in_class) + alpha * V)
            
            numerator = tfOLabel[emo][term] + smoothing
            denominator = totalTerm[emo] + (uniqTerm)
            if denominator == 0: # menghindari pembagian dengan 0
                cpDict[emo][term] = 0.0
            else:
                cpDict[emo][term] = numerator / denominator


### 5.2. MNB Training Function (`train`)

In [35]:
def train():
    global number_of_unique_term_across_label_full, number_of_unique_term_across_label_reff
    global total_label_frequency
    
    train_df = pd.read_csv(resultFilename[0], encoding='utf-8')
    print(f'Start training on {resultFilename[0]} with {len(train_df)} records.')

    for index, row in train_df.iterrows():
        full_lyrics = row['full']
        reff_lyrics = row['reff']
        emotion = row['emotion']

        processed_full = preProcess(full_lyrics)
        processed_reff = preProcess(reff_lyrics)
        
        countFreq(emotion, processed_full, tf_given_label_full, number_of_term_at_a_label_full)
        countFreq(emotion, processed_reff, tf_given_label_reff, number_of_term_at_a_label_reff)

        song_unique_term_full.update(set(processed_full))
        song_unique_term_reff.update(set(processed_reff))

        label_frequency[emotion] += 1

    number_of_unique_term_across_label_full = len(song_unique_term_full)
    number_of_unique_term_across_label_reff = len(song_unique_term_reff)

    total_label_frequency = sum(label_frequency.values())
    
    if total_label_frequency == 0:
        print("Warning: No training data seems to have been processed, or label_frequency is empty.")
        
    calcCP(song_unique_term_full, emotion_label, term_conditional_probability_at_label_full, tf_given_label_full, number_of_term_at_a_label_full, number_of_unique_term_across_label_full)
    calcCP(song_unique_term_reff, emotion_label, term_conditional_probability_at_label_reff, tf_given_label_reff, number_of_term_at_a_label_reff, number_of_unique_term_across_label_reff)
    
    print('Training finished')

### 5.3. MNB Testing Function (`test`)

In [36]:
def test():
    correct_full_predictions = 0
    correct_reff_predictions = 0
    total_songs_tested = 0
    
    y_true_fold_list = []
    y_pred_full_fold_list = []
    y_pred_reff_fold_list = []
    
    test_df = pd.read_csv(resultFilename[1], encoding='utf-8')

    for index, row in test_df.iterrows():
        total_songs_tested += 1
        full_lyrics = row['full']
        reff_lyrics = row['reff']
        actual_emotion = row['emotion']
        
        y_true_fold_list.append(actual_emotion)

        processed_full = preProcess(full_lyrics)
        processed_reff = preProcess(reff_lyrics)

        mnb_probabilities_full = defaultdict(float)
        mnb_probabilities_reff = defaultdict(float)

        current_song_terms_frequency_full = defaultdict(int)
        for term in processed_full:
            current_song_terms_frequency_full[term] += 1

        current_song_terms_frequency_reff = defaultdict(int)
        for term in processed_reff:
            current_song_terms_frequency_reff[term] += 1

        for current_emotion_candidate in emotion_label:
            
            class_count = label_frequency.get(current_emotion_candidate, 0)
            
            log_prior_prob = np.log(class_count) - np.log(total_label_frequency)
            
            log_likelihood_full = 0.0
            for term, freq in current_song_terms_frequency_full.items():
                term_prob = term_conditional_probability_at_label_full[current_emotion_candidate].get(term, 0.0)

                if term_prob == 0.0: 
                    N_c_full = number_of_term_at_a_label_full.get(current_emotion_candidate, 0)
                    V_full = number_of_unique_term_across_label_full
                    
                    denominator_full = N_c_full + (V_full * smoothing)
                    if denominator_full > 0:
                        term_prob = smoothing / denominator_full 
                    else:
                        term_prob = 1e-100 
                
                log_likelihood_full += freq * np.log(term_prob)
            mnb_probabilities_full[current_emotion_candidate] = log_prior_prob + log_likelihood_full

            log_likelihood_reff = 0.0
            for term, freq in current_song_terms_frequency_reff.items():
                term_prob = term_conditional_probability_at_label_reff[current_emotion_candidate].get(term, 0.0)

                if term_prob == 0.0:
                    N_c_reff = number_of_term_at_a_label_reff.get(current_emotion_candidate, 0)
                    V_reff = number_of_unique_term_across_label_reff
                    
                    denominator_reff = N_c_reff + (V_reff * smoothing)
                    if denominator_reff > 0:
                        term_prob = smoothing / denominator_reff
                    else:
                        term_prob = 1e-100
                
                log_likelihood_reff += freq * np.log(term_prob)
            mnb_probabilities_reff[current_emotion_candidate] = log_prior_prob + log_likelihood_reff

        predicted_emotion_full = ""
        if mnb_probabilities_full: 
            if all(v == -np.inf for v in mnb_probabilities_full.values()):
                predicted_emotion_full = emotion_label[0] if emotion_label else "unknown"
            else:
                predicted_emotion_full = max(mnb_probabilities_full, key=mnb_probabilities_full.get)
        else: 
            predicted_emotion_full = emotion_label[0] if emotion_label else "unknown"
        y_pred_full_fold_list.append(predicted_emotion_full)
        if predicted_emotion_full == actual_emotion:
            correct_full_predictions += 1

        predicted_emotion_reff = ""
        if mnb_probabilities_reff:
            if all(v == -np.inf for v in mnb_probabilities_reff.values()):
                predicted_emotion_reff = emotion_label[0] if emotion_label else "unknown"
            else:
                predicted_emotion_reff = max(mnb_probabilities_reff, key=mnb_probabilities_reff.get)
        else: 
            predicted_emotion_reff = emotion_label[0] if emotion_label else "unknown"
        y_pred_reff_fold_list.append(predicted_emotion_reff)
        if predicted_emotion_reff == actual_emotion:
            correct_reff_predictions += 1

    print(f"Testing finished. Full Correct: {correct_full_predictions}, Reff Correct: {correct_reff_predictions}, Total Tested: {total_songs_tested}")
    return correct_full_predictions, correct_reff_predictions, total_songs_tested, y_true_fold_list, y_pred_full_fold_list, y_pred_reff_fold_list

### 5.4. Custom Metrics Calculation Function
This function calculates per-emotion precision, recall, F1-score, and support, along with overall accuracy, without relying on scikit-learn.

In [37]:
def calculate_and_display_metrics(y_true_list, y_pred_list, labels, model_name="Model"):
    metrics_data = []
    if not y_true_list or not y_pred_list: # Handle empty lists
        print(f"No data to calculate metrics for {model_name}.")
        return pd.DataFrame(), 0.0
        
    for emotion in labels:
        tp = 0 # True Positives
        fp = 0 # False Positives
        fn = 0 # False Negatives
        # tn = 0 # True Negatives (less direct for multi-class, focusing on per-class P, R, F1)

        for true_label, pred_label in zip(y_true_list, y_pred_list):
            if true_label == emotion and pred_label == emotion:
                tp += 1
            elif true_label != emotion and pred_label == emotion:
                fp += 1
            elif true_label == emotion and pred_label != emotion:
                fn += 1
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        support = y_true_list.count(emotion)

        metrics_data.append({
            'Emotion': emotion,
            'Precision': f"{precision:.2f}",
            'Recall': f"{recall:.2f}",
            'F1-Score': f"{f1_score:.2f}",
            'Support (True)': support,
            'TP': tp,
            'FP': fp,
            'FN': fn
        })

    metrics_df = pd.DataFrame(metrics_data)
    print(f"\n<======== Detailed Metrics for {model_name} (Aggregated) =========>")
    print(metrics_df.to_string())

    correct_predictions = 0
    for i in range(len(y_true_list)):
        if y_true_list[i] == y_pred_list[i]:
            correct_predictions += 1
    
    overall_accuracy = correct_predictions / len(y_true_list) if len(y_true_list) > 0 else 0.0
    print(f"\nOverall {model_name} Accuracy: {overall_accuracy*100:.2f}% (based on {len(y_true_list)} predictions)")
    return metrics_df, overall_accuracy

## 6. K-Fold Cross-Validation Execution
Melakukan K-fold cross-validation. Di setiap iterasi (fold):
1. Beberapa fold akan dijadikan test set (sesuai `num_test_folds`), dan sisanya (K - `num_test_folds`) akan digabung menjadi training set.
2. Set ini akan dikonversi menjadi Pandas DataFrame dan disimpan sebagai `test-data.csv` dan `train-data.csv`.
3. Variabel global MNB direset.
4. Model di-train menggunakan `train()` (membaca `train-data.csv`).
5. Model dievaluasi menggunakan `test()` (membaca `test-data.csv`).
6. Skor akurasi dan prediksi diakumulasi.

Akhirnya, rata-rata akurasi dan metrik per-emosi akan ditampilkan.

In [ ]:
total_correct_full_overall = 0
total_correct_reff_overall = 0
total_samples_processed_overall = 0

overall_y_true = []
overall_y_pred_full = []
overall_y_pred_reff = []

num_test_folds = int(fold * dataSplit[1]) 

if num_test_folds >= fold:
    raise ValueError("num_test_folds must be less than the total number of initial divisions (fold variable).")
if num_test_folds <= 0:
    raise ValueError("num_test_folds must be at least 1.")

num_train_folds = fold - num_test_folds
print(f"Total initial data divisions (k_initial): {fold}")
print(f"Number of divisions for testing in each iteration: {num_test_folds}")
print(f"Number of divisions for training in each iteration: {num_train_folds}")
print(f"This will result in approximately a {num_train_folds*100/fold:.0f}% / {num_test_folds*100/fold:.0f}% train/test split per iteration.")

for i in range(fold): 
    print(f"\nIteration {i+1}/{fold}")
    
    train_data_for_current_iteration = []
    test_data_for_current_iteration = []

    current_test_fold_indices = []
    for j in range(num_test_folds):
        current_test_fold_indices.append((i + j) % fold)

    current_train_fold_indices = []
    for k_idx in range(fold):
        if k_idx not in current_test_fold_indices:
            current_train_fold_indices.append(k_idx)

    for fold_idx in current_test_fold_indices:
        if fold_idx in foldedList:
            test_data_for_current_iteration.extend(foldedList[fold_idx])

    for fold_idx in current_train_fold_indices:
        if fold_idx in foldedList:
            train_data_for_current_iteration.extend(foldedList[fold_idx])
    
    if train_data_for_current_iteration:
        train_df_current = pd.DataFrame(train_data_for_current_iteration, columns=column_csvs)
        train_df_current.to_csv(resultFilename[0], index=False, encoding='utf-8') 
    else:
        pd.DataFrame(columns=column_csvs).to_csv(resultFilename[0], index=False, encoding='utf-8')

    if test_data_for_current_iteration:
        test_df_current = pd.DataFrame(test_data_for_current_iteration, columns=column_csvs)
        test_df_current.to_csv(resultFilename[1], index=False, encoding='utf-8') 
    else:
        pd.DataFrame(columns=column_csvs).to_csv(resultFilename[1], index=False, encoding='utf-8')

    resetGlobals() 

    if train_data_for_current_iteration: 
        train()

    iter_correct_full, iter_correct_reff, iter_total_tested = 0, 0, 0
    iter_y_true, iter_y_pred_full, iter_y_pred_reff = [], [], []

    if test_data_for_current_iteration: 
        if train_data_for_current_iteration : 
            iter_correct_full, iter_correct_reff, iter_total_tested, iter_y_true, iter_y_pred_full, iter_y_pred_reff = test()
            
            overall_y_true.extend(iter_y_true)
            overall_y_pred_full.extend(iter_y_pred_full)
            overall_y_pred_reff.extend(iter_y_pred_reff)

    total_correct_full_overall += iter_correct_full
    total_correct_reff_overall += iter_correct_reff
    total_samples_processed_overall += iter_total_tested
    
    if iter_total_tested > 0:
        print(f"Iteration {i+1} Accuracy (Full): {iter_correct_full/iter_total_tested*100:.2f}%")
        print(f"Iteration {i+1} Accuracy (Reff): {iter_correct_reff/iter_total_tested*100:.2f}%")

print("\n<======== Cross-Validation Summary (Aggregated over all iterations) =========>")
if total_samples_processed_overall > 0:
    calculate_and_display_metrics(overall_y_true, overall_y_pred_full, emotion_label, model_name="Full Lyrics Model")
    calculate_and_display_metrics(overall_y_true, overall_y_pred_reff, emotion_label, model_name="Reff Lyrics Model")
else:
    print("No samples were processed across all iterations. Cannot calculate average accuracy or metrics.")

if fold > 0 and total_samples_processed_overall > 0:
    print("\n<======== Original Calculation Method (for comparison based on fixed avg test len) =========>")
    avg_correct_full_orig_calc_base = total_correct_full_overall / fold 
    print(f"Rata-rata kebenaran label (Full - orig calc base): {avg_correct_full_orig_calc_base:.2f} lagu penuh")

    avg_correct_reff_orig_calc_base = total_correct_reff_overall / fold
    print(f"Rata-rata kebenaran label (Reff - orig calc base): {avg_correct_reff_orig_calc_base:.2f} reff")

Total initial data divisions (k_initial): 10
Number of divisions for testing in each iteration: 2
Number of divisions for training in each iteration: 8
This will result in approximately a 80% / 20% train/test split per iteration.

Iteration 1/10
Start training on ./data/train-data.csv with 300 records.
Training finished
Testing finished. Full Correct: 48, Reff Correct: 36, Total Tested: 72
Iteration 1 Accuracy (Full): 66.67%
Iteration 1 Accuracy (Reff): 50.00%

Iteration 2/10
Start training on ./data/train-data.csv with 299 records.
Training finished
Testing finished. Full Correct: 62, Reff Correct: 42, Total Tested: 73
Iteration 2 Accuracy (Full): 84.93%
Iteration 2 Accuracy (Reff): 57.53%

Iteration 3/10
Start training on ./data/train-data.csv with 298 records.
Training finished
Testing finished. Full Correct: 70, Reff Correct: 49, Total Tested: 74
Iteration 3 Accuracy (Full): 94.59%
Iteration 3 Accuracy (Reff): 66.22%

Iteration 4/10
Start training on ./data/train-data.csv with 298 

## 7. Function untuk Prediksi Berbasis Input

In [39]:
def testLy(lyrics):
    lyr_token = preProcess(lyrics)

    mnb_prob = defaultdict(float)
    current_song_term_frequency = defaultdict(int)

    for term in lyr_token:
        current_song_term_frequency[term] += 1

    for current_emotion_candidate in emotion_label:
        log_prior_prob = np.log(label_frequency[current_emotion_candidate]) - np.log(total_label_frequency)
        
        log_likelihood = 0.0
        for term, freq in current_song_term_frequency.items():
            term_prob = term_conditional_probability_at_label_full[current_emotion_candidate].get(term, 0.0)
            if term_prob > 0:
                log_likelihood += freq * np.log(term_prob)
            else:
                denominator = (number_of_term_at_a_label_full.get(current_emotion_candidate, 0) + (number_of_unique_term_across_label_full * smoothing))
                if denominator > 0:
                    smoothed_unseen_prob = smoothing / denominator
                    if smoothed_unseen_prob > 0:
                        log_likelihood += freq * np.log(smoothed_unseen_prob)
                    else:
                        log_likelihood += -np.inf
                else:
                    log_likelihood += -np.inf
        mnb_prob[current_emotion_candidate] = log_prior_prob + log_likelihood

    probsNames = []
    for key in mnb_prob:
        probsNames.append(key)

    probsNames.sort(key = mnb_prob.get)

    predicted_emotion = max(mnb_prob, key=mnb_prob.get)
    return predicted_emotion, mnb_prob, probsNames

lirik_bos = "Berat bebanku Meninggalkanmu Separuh nafas jiwaku sirna Bukan salahmu Apa dayaku Mungkin benar cinta sejati tak berpihak pada kita Kasihku Sampai disini kisah kita Jangan tangisi keadaannya Bukan karena kita berbeda Dengarkan Dengarkan lagu lagu ini Melodi rintihan hati ini Kisah kita Berakhir di Januari Selamat tinggal kisah sejatiku Wow pergilah Kasihku Sampai disini kisah kita Jangan tangisi keadaannya Bukan karena kita berbeda Dengarkan lagu lagu ini Melodi rintihan hati ini Kisah kita berakhir di januari Dengarkan lagu lagu ini Melodi rintihan hati ini Kisah kita berakhir Berakhir Berakhir di januari Berakhir di januari"

result, probabilities, sorted = testLy(lirik_bos)
print(testLy(lirik_bos))
print("Prediction From Most Confident to Least:")

('sedih', defaultdict(<class 'float'>, {'bahagia': np.float64(-481.3825823888632), 'sedih': np.float64(-391.240593598259), 'marah': np.float64(-526.9746566187333), 'takut': np.float64(-563.7782222149358)}), ['takut', 'marah', 'bahagia', 'sedih'])
Prediction From Most Confident to Least:
